# Synthetic Data Generation for VoltStream EV OPE Demo

## Business Objective

Generate causally-linked synthetic data that demonstrates the OEE vs OPE gap problem in EV manufacturing. The data tells a compelling story where environmental conditions (low humidity → high dust) cause AGV sensor failures, leading to material starvation and OPE drops.

## Technical Approach

Using Snowpark Python, we generate data directly in Snowflake using `session.write_pandas()` for efficient bulk loading. This approach is faster than CSV file generation and upload.

## Data Story

- **Months 1-2 (Baseline)**: Steady production, normal humidity (~45%), low dust (~10 µg/m³), 99.9% AGV reliability
- **Month 3 Week 2 (Crisis)**: Humidity drops to 25% → Dust spikes to 35 µg/m³ → AGV-ERR-99 sensor errors → Line starvation → OPE drops to 60%

## Notebook Structure

| Section | Purpose |
|---------|--------|
| 1. Environment Setup | Import packages, connect to Snowflake |
| 2. Configuration | Define date ranges, constants, parameters |
| 3. Helper Functions | Utility functions for data generation |
| 4. Generate Dimension Data | Create ASSET and PRODUCT dimensions |
| 5. Generate Environmental Data | Humidity, dust, temperature sensors |
| 6. Generate AGV Telemetry | AGV positions and error codes |
| 7. Generate Equipment States | Downtime and starvation events |
| 8. Generate Production Orders | Work orders with actual quantities |
| 9. Generate Material Documents | Inventory movements |
| 10. Verification | Row counts and data quality checks |

## Output

- `ATOMIC.ASSET` - 25 assets (5 lines + 20 AGVs)
- `ATOMIC.PRODUCT` - 6 products (battery packs, modules, cells)
- `RAW.ENV_SENSOR_STREAM_STAGE` - ~1.5M environmental readings
- `RAW.AGV_TELEMATICS_LOG_CDC` - ~2.6M AGV telemetry records
- `RAW.EQUIPMENT_STATE_LOG_CDC` - ~11K equipment state events
- `RAW.PROD_ORDER_HEADER_CDC` - ~1.4K production orders
- `RAW.MATERIAL_DOCUMENT_CDC` - ~7K material movements


## Learning Objectives

After completing this notebook, you will understand:
1. How to generate causally-linked synthetic manufacturing data
2. The relationship between environmental conditions and equipment failures
3. How to use Snowpark `write_pandas()` for efficient bulk loading
4. Patterns for creating time-series data with controlled anomaly periods

## Prerequisites

- **Python**: Familiarity with pandas, numpy, and datetime operations
- **Snowflake**: Basic understanding of Snowpark sessions and table operations
- **Domain**: General understanding of manufacturing OEE/OPE concepts

## Idempotency

This notebook is **idempotent** - it can be executed multiple times safely.
All tables are truncated at the start before data generation begins.


## 1. Environment Setup


In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import numpy as np
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta
from snowflake.snowpark.context import get_active_session

# Visualization imports
import matplotlib.pyplot as plt

print("✅ All packages imported")


In [ ]:
# =============================================================================
# SNOWFLAKE SESSION
# =============================================================================

session = get_active_session()

# Verify connection
result = session.sql("SELECT CURRENT_DATABASE(), CURRENT_SCHEMA(), CURRENT_WAREHOUSE()").collect()
print(f"✅ Connected to Snowflake")
print(f"   Database: {result[0][0]}")
print(f"   Schema: {result[0][1]}")
print(f"   Warehouse: {result[0][2]}")


In [ ]:
# =============================================================================
# CLEANUP - Truncate tables to ensure clean state on re-run
# =============================================================================

print("Truncating tables for clean data generation...")

# Truncate RAW tables (will be populated by this notebook)
session.sql("TRUNCATE TABLE IF EXISTS RAW.ENV_SENSOR_STREAM_STAGE").collect()
session.sql("TRUNCATE TABLE IF EXISTS RAW.AGV_TELEMATICS_LOG_CDC").collect()
session.sql("TRUNCATE TABLE IF EXISTS RAW.EQUIPMENT_STATE_LOG_CDC").collect()
session.sql("TRUNCATE TABLE IF EXISTS RAW.PROD_ORDER_HEADER_CDC").collect()
session.sql("TRUNCATE TABLE IF EXISTS RAW.MATERIAL_DOCUMENT_CDC").collect()

# Truncate ATOMIC dimension tables (will be populated by this notebook)
session.sql("TRUNCATE TABLE IF EXISTS ATOMIC.ASSET").collect()
session.sql("TRUNCATE TABLE IF EXISTS ATOMIC.PRODUCT").collect()

print("✅ Tables truncated - ready for fresh data generation")


## 2. Configuration


In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Date range: 3 months of data
START_DATE = datetime(2024, 10, 1)
END_DATE = datetime(2024, 12, 31)

# Crisis period: Month 3, Week 2 (Dec 9-11, 2024)
CRISIS_START = datetime(2024, 12, 9)
CRISIS_END = datetime(2024, 12, 11, 23, 59, 59)

# Production lines
PRODUCTION_LINES = ['LINE_1', 'LINE_2', 'LINE_3', 'LINE_4', 'LINE_5']

# AGVs (20 total)
AGVS = [f'AGV_{i:03d}' for i in range(1, 21)]

# Zones
ZONES = ['ZONE_A', 'ZONE_B', 'ZONE_C', 'ZONE_D']

# Shifts
SHIFTS = ['SHIFT_1', 'SHIFT_2', 'SHIFT_3']

# Products (Battery hierarchy)
PRODUCTS = [
    {'sku': 'BP-100', 'name': 'Battery Pack 100kWh', 'category': 'BATTERY_PACK'},
    {'sku': 'BP-150', 'name': 'Battery Pack 150kWh', 'category': 'BATTERY_PACK'},
    {'sku': 'MOD-20', 'name': 'Battery Module 20kWh', 'category': 'MODULE'},
    {'sku': 'MOD-25', 'name': 'Battery Module 25kWh', 'category': 'MODULE'},
    {'sku': 'CELL-2170', 'name': '2170 Cell', 'category': 'CELL'},
    {'sku': 'CELL-4680', 'name': '4680 Cell', 'category': 'CELL'},
]

print("✅ Configuration loaded")
print(f"   Date range: {START_DATE.date()} to {END_DATE.date()} ({(END_DATE - START_DATE).days + 1} days)")
print(f"   Crisis period: {CRISIS_START.date()} to {CRISIS_END.date()}")
print(f"   Production lines: {len(PRODUCTION_LINES)}")
print(f"   AGVs: {len(AGVS)}")


## 3. Helper Functions


In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def is_crisis_period(timestamp: datetime) -> bool:
    """Check if timestamp falls within the crisis period."""
    return CRISIS_START <= timestamp <= CRISIS_END


def get_shift_for_hour(hour: int) -> str:
    """Get shift ID based on hour of day."""
    if 6 <= hour < 14:
        return 'SHIFT_1'
    elif 14 <= hour < 22:
        return 'SHIFT_2'
    else:
        return 'SHIFT_3'


def generate_uuid() -> str:
    """Generate a UUID string."""
    return str(uuid.uuid4())


def convert_timestamps(df: pd.DataFrame) -> pd.DataFrame:
    """Convert all datetime columns to string format for Snowflake compatibility."""
    df = df.copy()
    for col in df.columns:
        # Check if column contains datetime objects
        if len(df) > 0:
            non_null = df[col].dropna()
            if len(non_null) > 0:
                sample = non_null.iloc[0]
                if isinstance(sample, datetime):
                    # Use pd.notna() to properly handle NaT values
                    df[col] = df[col].apply(
                        lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notna(x) else None
                    )
    return df


def write_to_snowflake(df: pd.DataFrame, table_name: str, schema: str = 'RAW'):
    """
    Write DataFrame to Snowflake table using write_pandas.
    FAIL-FAST: Raises exception on failure.
    """
    # Get database name and strip any quotes
    db_name = session.get_current_database().replace('"', '')
    full_table_name = f"{schema}.{table_name}"
    
    # Convert to uppercase column names for Snowflake compatibility
    df.columns = [col.upper() for col in df.columns]
    
    # Convert datetime columns to string format for Snowflake compatibility
    df = convert_timestamps(df)
    
    # Write to Snowflake (append mode - tables are truncated at start of notebook)
    snowpark_df = session.write_pandas(
        df,
        table_name=table_name,
        database=db_name,
        schema=schema,
        auto_create_table=False,
        overwrite=False
    )
    
    print(f"   ✅ Wrote {len(df):,} rows to {full_table_name}")
    return snowpark_df


print("✅ Helper functions defined")


## 4. Generate Dimension Data

Create master data for assets (production lines and AGVs) and products.


In [ ]:
# =============================================================================
# GENERATE ASSET DIMENSION
# =============================================================================

print("Generating ASSET dimension...")

asset_records = []
asset_id = 1

# Production lines
for line in PRODUCTION_LINES:
    asset_records.append({
        'ASSET_ID': asset_id,
        'ASSET_CODE': line,
        'ASSET_NAME': f'Production Line {line[-1]}',
        'ASSET_DESCRIPTION': f'Battery pack assembly line {line[-1]}',
        'ASSET_TYPE': 'PRODUCTION_LINE',
        'ASSET_CLASS': 'ASSEMBLY',
        'ASSET_STATUS': 'ACTIVE',
        'SITE_ID': 1,
        'PRODUCTION_LINE_ID': line,
        'ZONE_ID': f'ZONE_{chr(64 + int(line[-1]))}',
        'MANUFACTURER': 'Siemens',
        'MODEL_NUMBER': 'SIMATIC-5000'
    })
    asset_id += 1

# AGVs
for agv in AGVS:
    asset_records.append({
        'ASSET_ID': asset_id,
        'ASSET_CODE': agv,
        'ASSET_NAME': f'Automated Guided Vehicle {agv[-3:]}',
        'ASSET_DESCRIPTION': 'Material transport AGV with LiDAR navigation',
        'ASSET_TYPE': 'AGV',
        'ASSET_CLASS': 'TRANSPORT',
        'ASSET_STATUS': 'ACTIVE',
        'SITE_ID': 1,
        'PRODUCTION_LINE_ID': None,
        'ZONE_ID': random.choice(ZONES),
        'MANUFACTURER': 'KUKA',
        'MODEL_NUMBER': 'KMR-iiwa'
    })
    asset_id += 1

df_assets = pd.DataFrame(asset_records)

# Write to ATOMIC.ASSET
write_to_snowflake(df_assets, 'ASSET', 'ATOMIC')
print(f"   Total assets: {len(df_assets)} ({len(PRODUCTION_LINES)} lines + {len(AGVS)} AGVs)")


In [ ]:
# =============================================================================
# GENERATE PRODUCT DIMENSION
# =============================================================================

print("Generating PRODUCT dimension...")

product_records = []
for i, product in enumerate(PRODUCTS, 1):
    product_records.append({
        'PRODUCT_ID': i,
        'PRODUCT_CODE': product['sku'],
        'PRODUCT_NAME': product['name'],
        'PRODUCT_DESCRIPTION_SHORT': f"High-performance {product['name'].lower()} for electric vehicles",
        'PRODUCT_CATEGORY': product['category'],
        'PRODUCT_FAMILY': 'EV_BATTERY',
        'PRODUCT_LINE': 'VOLTSTREAM',
        'UNIT_OF_MEASURE': 'EA',
        'STANDARD_COST': round(random.uniform(100, 5000), 2),
        'STANDARD_CYCLE_TIME_SEC': random.randint(30, 120)
    })

df_products = pd.DataFrame(product_records)

# Write to ATOMIC.PRODUCT
write_to_snowflake(df_products, 'PRODUCT', 'ATOMIC')
print(f"   Total products: {len(df_products)}")


## 5. Generate Environmental Sensor Data

Environmental readings at 5-minute intervals. During the crisis period:
- Humidity drops from ~45% to ~25%
- Dust (PM2.5) increases from ~10 to ~35 µg/m³


In [ ]:
# =============================================================================
# GENERATE ENVIRONMENTAL SENSOR DATA
# =============================================================================

print("Generating ENV_SENSOR_STREAM_STAGE...")
print("   This may take a few minutes for ~1.5M rows...")

# Reset seed for reproducibility
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

records = []
current_time = START_DATE
reading_interval = timedelta(minutes=5)
sensor_types = ['HUMIDITY', 'PM25', 'TEMPERATURE']
batch_size = 100000
total_rows = 0

while current_time <= END_DATE:
    is_crisis = is_crisis_period(current_time)
    
    for zone in ZONES:
        for line in PRODUCTION_LINES:
            for sensor_type in sensor_types:
                sensor_id = f"{zone}_{line}_{sensor_type}_01"
                
                if sensor_type == 'HUMIDITY':
                    if is_crisis:
                        value = max(15, min(40, np.random.normal(25, 3)))
                    else:
                        value = max(30, min(60, np.random.normal(45, 5)))
                    unit = '%'
                elif sensor_type == 'PM25':
                    if is_crisis:
                        base_dust = 50 - 25 + np.random.normal(10, 5)
                        value = max(5, min(50, base_dust))
                    else:
                        value = max(2, min(20, np.random.normal(10, 2)))
                    unit = 'µg/m³'
                else:  # TEMPERATURE
                    value = np.random.normal(22, 1.5)
                    unit = '°C'
                
                records.append({
                    'READING_ID': generate_uuid(),
                    'SENSOR_ID': sensor_id,
                    'SENSOR_TYPE': sensor_type,
                    'ZONE_ID': zone,
                    'PRODUCTION_LINE_ID': line,
                    'READING_TIMESTAMP': current_time,
                    'METRIC_NAME': sensor_type,
                    'METRIC_VALUE': round(value, 3),
                    'UNIT_OF_MEASURE': unit,
                    'QUALITY_FLAG': 'GOOD'
                })
    
    # Write in batches to avoid memory issues
    if len(records) >= batch_size:
        df = pd.DataFrame(records)
        write_to_snowflake(df, 'ENV_SENSOR_STREAM_STAGE', 'RAW')
        total_rows += len(records)
        records = []
    
    current_time += reading_interval

# Write remaining records
if records:
    df = pd.DataFrame(records)
    write_to_snowflake(df, 'ENV_SENSOR_STREAM_STAGE', 'RAW')
    total_rows += len(records)

print(f"✅ Total ENV_SENSOR_STREAM_STAGE rows: {total_rows:,}")


## 6. Generate AGV Telemetry Data

AGV position and error data at 1-minute intervals. During crisis:
- 15% error probability when dust is high
- AGV-ERR-99 (optical sensor obscured) is the primary error


In [ ]:
# =============================================================================
# GENERATE AGV TELEMETRY DATA
# =============================================================================

print("Generating AGV_TELEMATICS_LOG_CDC...")
print("   This may take several minutes for ~2.6M rows...")

# Reset seed for reproducibility
random.seed(RANDOM_SEED + 1)
np.random.seed(RANDOM_SEED + 1)

records = []
current_time = START_DATE
reading_interval = timedelta(minutes=1)
batch_size = 100000
total_rows = 0

while current_time <= END_DATE:
    is_crisis = is_crisis_period(current_time)
    
    for agv_id in AGVS:
        x = random.uniform(0, 500)
        y = random.uniform(0, 300)
        zone = random.choice(ZONES)
        route = f"ROUTE_{random.randint(1, 10):02d}"
        velocity = random.uniform(0.5, 2.0) if random.random() > 0.1 else 0
        battery = random.uniform(20, 100)
        
        error_code = None
        error_severity = None
        error_message = None
        
        if is_crisis:
            if random.random() < 0.15:
                error_code = 'AGV-ERR-99'
                error_severity = 'ERROR'
                error_message = 'Optical sensor obscured - reduced visibility detected'
        else:
            if random.random() < 0.001:
                error_code = random.choice(['AGV-ERR-01', 'AGV-ERR-42', 'AGV-ERR-55'])
                error_severity = 'WARNING'
                error_message = 'Minor operational issue detected'
        
        records.append({
            'MSG_ID': generate_uuid(),
            'AGV_ID': agv_id,
            'EVENT_TIMESTAMP': current_time,
            'X_COORD': round(x, 3),
            'Y_COORD': round(y, 3),
            'Z_COORD': 0,
            'ZONE_ID': zone,
            'ROUTE_SEGMENT': route,
            'VELOCITY': round(velocity, 3),
            'BATTERY_LEVEL': round(battery, 2),
            'PAYLOAD_ID': f'BATCH_{random.randint(1000, 9999)}' if random.random() > 0.3 else None,
            'ERROR_CODE': error_code,
            'ERROR_SEVERITY': error_severity,
            'ERROR_MESSAGE': error_message,
            '_CDC_OPERATION': 'INSERT',
            '_CDC_TIMESTAMP': current_time,
            '_CDC_SEQUENCE': int(current_time.timestamp() * 1000),
            '_CDC_SOURCE_SYSTEM': 'SIEMENS_MES'
        })
    
    if len(records) >= batch_size:
        df = pd.DataFrame(records)
        write_to_snowflake(df, 'AGV_TELEMATICS_LOG_CDC', 'RAW')
        total_rows += len(records)
        records = []
    
    current_time += reading_interval

if records:
    df = pd.DataFrame(records)
    write_to_snowflake(df, 'AGV_TELEMATICS_LOG_CDC', 'RAW')
    total_rows += len(records)

print(f"✅ Total AGV_TELEMATICS_LOG_CDC rows: {total_rows:,}")


## 7. Generate Equipment State Data

Equipment state transitions at hourly intervals. During crisis:
- LINE_4 has 40% chance of STARVATION state
- Other lines have 20% chance of STARVATION


In [ ]:
# =============================================================================
# GENERATE EQUIPMENT STATE LOG
# =============================================================================
# 
# Updated logic to create visible OEE-OPE gap:
# - Normal period: 85% running, 10.5% baseline starvation, 4.5% other stops
# - Crisis LINE_4: 60% starvation with 15-40 min duration
# - Crisis other lines: 35% starvation with 10-20 min duration

print("Generating EQUIPMENT_STATE_LOG_CDC...")

# Reset seed for reproducibility
random.seed(RANDOM_SEED + 2)
np.random.seed(RANDOM_SEED + 2)

records = []
current_date = START_DATE.date()

while current_date <= END_DATE.date():
    current_datetime = datetime.combine(current_date, datetime.min.time())
    is_crisis = is_crisis_period(current_datetime)
    
    for line in PRODUCTION_LINES:
        for hour in range(24):
            event_time = current_datetime + timedelta(hours=hour)
            shift_id = get_shift_for_hour(hour)
            
            if is_crisis and line == 'LINE_4':
                # Crisis LINE_4: 60% starvation with 15-40 min duration (severe impact)
                if random.random() < 0.60:
                    state_code = 2
                    state_name = 'IDLE'
                    reason_code = 'STARVATION'
                    reason_desc = 'Waiting for material - AGV delivery delayed due to sensor errors'
                    duration = random.randint(900, 2400)  # 15-40 minutes
                else:
                    state_code = 1
                    state_name = 'RUNNING'
                    reason_code = None
                    reason_desc = None
                    duration = 3600
            elif is_crisis:
                # Crisis other lines: 35% starvation with 10-20 min duration
                if random.random() < 0.35:
                    state_code = 2
                    state_name = 'IDLE'
                    reason_code = 'STARVATION'
                    reason_desc = 'Waiting for material - AGV delivery delayed'
                    duration = random.randint(600, 1200)  # 10-20 minutes
                else:
                    state_code = 1
                    state_name = 'RUNNING'
                    reason_code = None
                    reason_desc = None
                    duration = 3600
            else:
                # Normal period: Add baseline starvation for realistic OEE-OPE gap
                # 85% running, 10.5% baseline starvation, 4.5% other stops
                rand_val = random.random()
                if rand_val < 0.85:
                    # 85% fully running
                    state_code = 1
                    state_name = 'RUNNING'
                    reason_code = None
                    reason_desc = None
                    duration = 3600
                elif rand_val < 0.955:
                    # 10.5% baseline starvation (normal AGV cycle delays)
                    state_code = 2
                    state_name = 'IDLE'
                    reason_code = 'STARVATION'
                    reason_desc = 'Waiting for material - normal AGV cycle time'
                    duration = random.randint(180, 600)  # 3-10 minutes
                else:
                    # 4.5% other minor stops
                    state_code = random.choice([2, 3, 4])
                    state_name = ['', 'RUNNING', 'IDLE', 'FAULT', 'SETUP'][state_code]
                    reason_code = random.choice(['CHANGEOVER', 'MINOR_STOP', 'ADJUSTMENT'])
                    reason_desc = 'Planned minor stop'
                    duration = random.randint(60, 300)
            
            records.append({
                'EVENT_ID': generate_uuid(),
                'ASSET_ID': line,  # Must match ASSET_CODE in ATOMIC.ASSET for JOIN
                'PRODUCTION_LINE_ID': line,
                'EVENT_TIMESTAMP': event_time,
                'STATE_CODE': state_code,
                'STATE_NAME': state_name,
                'REASON_CODE': reason_code,
                'REASON_DESCRIPTION': reason_desc,
                'DURATION_SECONDS': duration,
                'SHIFT_ID': shift_id,
                'OPERATOR_ID': f'OP_{random.randint(100, 999)}',
                '_CDC_OPERATION': 'INSERT',
                '_CDC_TIMESTAMP': event_time,
                '_CDC_SEQUENCE': int(event_time.timestamp() * 1000),
                '_CDC_SOURCE_SYSTEM': 'SIEMENS_MES'
            })
    
    current_date += timedelta(days=1)

df_equip = pd.DataFrame(records)
write_to_snowflake(df_equip, 'EQUIPMENT_STATE_LOG_CDC', 'RAW')
print(f"✅ Total EQUIPMENT_STATE_LOG_CDC rows: {len(df_equip):,}")


## 8. Generate Production Orders

Production orders with realistic completion rates. **Key fix**: Includes ACTUAL_QTY column to show actual production vs target.


In [ ]:
# =============================================================================
# GENERATE PRODUCTION ORDERS
# =============================================================================

print("Generating PROD_ORDER_HEADER_CDC...")

# Reset seed for reproducibility
random.seed(RANDOM_SEED + 3)
np.random.seed(RANDOM_SEED + 3)

records = []
order_id = 1000000
current_date = START_DATE.date()

while current_date <= END_DATE.date():
    for line in PRODUCTION_LINES:
        for shift in SHIFTS:
            product = random.choice(PRODUCTS)
            target_qty = random.randint(50, 200)
            
            current_datetime = datetime.combine(current_date, datetime.min.time())
            is_crisis = is_crisis_period(current_datetime)
            
            # Determine completion rate based on crisis status
            if is_crisis and line == 'LINE_4':
                completion_rate = random.uniform(0.6, 0.7)  # 60-70% during crisis
            elif is_crisis:
                completion_rate = random.uniform(0.8, 0.9)  # 80-90% during crisis
            else:
                completion_rate = random.uniform(0.95, 1.0)  # 95-100% normal
            
            # Calculate actual quantity produced (KEY FIX)
            actual_qty = int(target_qty * completion_rate)
            
            shift_start_hour = {'SHIFT_1': 6, 'SHIFT_2': 14, 'SHIFT_3': 22}[shift]
            sched_start = datetime.combine(current_date, datetime.min.time()) + timedelta(hours=shift_start_hour)
            sched_end = sched_start + timedelta(hours=8)
            
            records.append({
                'ORDER_ID': f'PO{order_id}',
                'ORDER_TYPE': 'PP01',
                'SKU': product['sku'],
                'TARGET_QTY': target_qty,
                'ACTUAL_QTY': actual_qty,  # NEW: Store actual production
                'UNIT_OF_MEASURE': 'EA',
                'SCHED_START': sched_start,
                'SCHED_END': sched_end,
                'ACTUAL_START': sched_start,
                'ACTUAL_END': sched_end if completion_rate > 0.9 else None,
                'PRODUCTION_LINE_ID': line,
                'WORK_CENTER_ID': f'WC_{line}',
                'ORDER_STATUS': 'COMPLETED' if completion_rate > 0.9 else 'STARTED',
                'PRIORITY': random.randint(1, 5),
                '_CDC_OPERATION': 'INSERT',
                '_CDC_TIMESTAMP': sched_start,
                '_CDC_SEQUENCE': order_id,
                '_CDC_SOURCE_SYSTEM': 'SAP_ERP'
            })
            
            order_id += 1
    
    current_date += timedelta(days=1)

df_orders = pd.DataFrame(records)
write_to_snowflake(df_orders, 'PROD_ORDER_HEADER_CDC', 'RAW')
print(f"✅ Total PROD_ORDER_HEADER_CDC rows: {len(df_orders):,}")

# Calculate and display crisis period metrics
crisis_orders = df_orders[df_orders['ORDER_STATUS'] != 'COMPLETED']
if len(crisis_orders) > 0:
    avg_completion = crisis_orders['ACTUAL_QTY'].sum() / crisis_orders['TARGET_QTY'].sum() * 100
    print(f"   Crisis period avg completion rate: {avg_completion:.1f}%")


## 9. Generate Material Documents

Inventory movements with increased quality inspection during crisis period.


In [ ]:
# =============================================================================
# GENERATE MATERIAL DOCUMENTS
# =============================================================================

print("Generating MATERIAL_DOCUMENT_CDC...")

# Reset seed for reproducibility
random.seed(RANDOM_SEED + 4)
np.random.seed(RANDOM_SEED + 4)

records = []
doc_id = 5000000
current_date = START_DATE.date()

while current_date <= END_DATE.date():
    current_datetime = datetime.combine(current_date, datetime.min.time())
    is_crisis = is_crisis_period(current_datetime)
    
    for _ in range(random.randint(50, 100)):
        product = random.choice(PRODUCTS)
        batch_id = f'BATCH_{random.randint(10000, 99999)}'
        
        if is_crisis and random.random() < 0.3:
            stock_type = 'QUALITY_INSPECTION'
            mvmt_type = '321'
        else:
            stock_type = 'UNRESTRICTED'
            mvmt_type = random.choice(['101', '261', '601'])
        
        posting_time = current_datetime + timedelta(
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59)
        )
        
        records.append({
            'MAT_DOC_ID': str(doc_id),
            'POSTING_DATE': current_date,
            'SKU': product['sku'],
            'MVMT_TYPE': mvmt_type,
            'STOCK_TYPE': stock_type,
            'BATCH_ID': batch_id,
            'PLANT_ID': 'P001',
            'STORAGE_LOCATION': random.choice(['SL01', 'SL02', 'SL03']),
            'QUANTITY': random.randint(10, 500),
            'UNIT_OF_MEASURE': 'EA',
            '_CDC_OPERATION': 'INSERT',
            '_CDC_TIMESTAMP': posting_time,
            '_CDC_SEQUENCE': doc_id,
            '_CDC_SOURCE_SYSTEM': 'SAP_ERP'
        })
        
        doc_id += 1
    
    current_date += timedelta(days=1)

df_matdocs = pd.DataFrame(records)
write_to_snowflake(df_matdocs, 'MATERIAL_DOCUMENT_CDC', 'RAW')
print(f"✅ Total MATERIAL_DOCUMENT_CDC rows: {len(df_matdocs):,}")


## 10. Verification

Verify row counts and data quality.


In [ ]:
# =============================================================================
# VERIFICATION
# =============================================================================

print("\n" + "="*60)
print("DATA GENERATION COMPLETE")
print("="*60)

# Query row counts
verification_sql = """
SELECT 'ATOMIC.ASSET' AS TABLE_NAME, COUNT(*) AS ROW_COUNT FROM ATOMIC.ASSET
UNION ALL SELECT 'ATOMIC.PRODUCT', COUNT(*) FROM ATOMIC.PRODUCT
UNION ALL SELECT 'RAW.ENV_SENSOR_STREAM_STAGE', COUNT(*) FROM RAW.ENV_SENSOR_STREAM_STAGE
UNION ALL SELECT 'RAW.AGV_TELEMATICS_LOG_CDC', COUNT(*) FROM RAW.AGV_TELEMATICS_LOG_CDC
UNION ALL SELECT 'RAW.EQUIPMENT_STATE_LOG_CDC', COUNT(*) FROM RAW.EQUIPMENT_STATE_LOG_CDC
UNION ALL SELECT 'RAW.PROD_ORDER_HEADER_CDC', COUNT(*) FROM RAW.PROD_ORDER_HEADER_CDC
UNION ALL SELECT 'RAW.MATERIAL_DOCUMENT_CDC', COUNT(*) FROM RAW.MATERIAL_DOCUMENT_CDC
ORDER BY TABLE_NAME
"""

counts = session.sql(verification_sql).to_pandas()
print("\n📊 Row Counts:")
for _, row in counts.iterrows():
    print(f"   {row['TABLE_NAME']}: {row['ROW_COUNT']:,}")

# Verify crisis period data
crisis_sql = """
SELECT 
    DATE(EVENT_TIMESTAMP) AS EVENT_DATE,
    COUNT(*) AS TOTAL_READINGS,
    SUM(CASE WHEN ERROR_CODE = 'AGV-ERR-99' THEN 1 ELSE 0 END) AS ERR_99_COUNT
FROM RAW.AGV_TELEMATICS_LOG_CDC
WHERE DATE(EVENT_TIMESTAMP) BETWEEN '2024-12-08' AND '2024-12-12'
GROUP BY 1
ORDER BY 1
"""

crisis_data = session.sql(crisis_sql).to_pandas()
print("\n🔥 Crisis Period AGV Errors (Dec 8-12):")
for _, row in crisis_data.iterrows():
    print(f"   {row['EVENT_DATE']}: {row['ERR_99_COUNT']:,} AGV-ERR-99 errors")

print("\n✅ Data generation complete! Run 03b_raw_to_atomic.sql to populate ATOMIC tables.")


## 11. Data Quality Visualizations

Visual verification that the crisis period patterns were generated correctly.
These plots confirm the causal chain: Low Humidity → High Dust → AGV Errors → Starvation.


In [ ]:
# =============================================================================
# VISUALIZATION SETUP - Snowflake Dark Theme
# =============================================================================

import matplotlib.pyplot as plt

# Apply dark theme
plt.style.use('dark_background')
plt.rcParams.update({
    # Background colors (soft dark gray)
    'figure.facecolor': '#121212',
    'axes.facecolor': '#121212',
    
    # Text colors (off-white to reduce glare)
    'text.color': '#E5E5E7',
    'axes.labelcolor': '#E5E5E7',
    'xtick.color': '#A1A1A6',
    'ytick.color': '#A1A1A6',
    
    # Grid and axes (subtle)
    'axes.edgecolor': '#3A3A3C',
    'grid.color': '#2C2C2E',
    'grid.alpha': 0.6,
    
    # Figure quality
    'figure.dpi': 150,
    'savefig.dpi': 200,
    'figure.figsize': (14, 4),
    
    # Typography
    'font.family': 'sans-serif',
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
})

# Snowflake brand colors
SNOWFLAKE_BLUE = '#29B5E8'
MID_BLUE = '#11567F'
VALENCIA_ORANGE = '#FF9F0A'
ACCENT_CYAN = '#5AC8FA'

print("✅ Snowflake dark theme applied")


In [ ]:
# =============================================================================
# CRISIS PERIOD VERIFICATION PLOTS
# =============================================================================

# Query environmental data for crisis period (Dec 8-12)
env_sql = """
SELECT 
    DATE(READING_TIMESTAMP) AS READING_DATE,
    SENSOR_TYPE,
    AVG(METRIC_VALUE) AS AVG_VALUE
FROM RAW.ENV_SENSOR_STREAM_STAGE
WHERE DATE(READING_TIMESTAMP) BETWEEN '2024-12-06' AND '2024-12-14'
  AND SENSOR_TYPE IN ('HUMIDITY', 'PM25')
GROUP BY 1, 2
ORDER BY 1, 2
"""
env_df = session.sql(env_sql).to_pandas()

# Query AGV errors
agv_sql = """
SELECT 
    DATE(EVENT_TIMESTAMP) AS EVENT_DATE,
    COUNT(*) AS TOTAL_READINGS,
    SUM(CASE WHEN ERROR_CODE = 'AGV-ERR-99' THEN 1 ELSE 0 END) AS ERR_99_COUNT,
    ROUND(100.0 * SUM(CASE WHEN ERROR_CODE = 'AGV-ERR-99' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ERROR_RATE_PCT
FROM RAW.AGV_TELEMATICS_LOG_CDC
WHERE DATE(EVENT_TIMESTAMP) BETWEEN '2024-12-06' AND '2024-12-14'
GROUP BY 1
ORDER BY 1
"""
agv_df = session.sql(agv_sql).to_pandas()

# Query production completion by line during crisis
prod_sql = """
SELECT 
    PRODUCTION_LINE_ID,
    CASE 
        WHEN DATE(SCHED_START) BETWEEN '2024-12-09' AND '2024-12-11' THEN 'Crisis'
        ELSE 'Normal'
    END AS PERIOD,
    ROUND(100.0 * SUM(ACTUAL_QTY) / NULLIF(SUM(TARGET_QTY), 0), 1) AS COMPLETION_RATE
FROM RAW.PROD_ORDER_HEADER_CDC
WHERE DATE(SCHED_START) BETWEEN '2024-12-01' AND '2024-12-14'
GROUP BY 1, 2
ORDER BY 1, 2
"""
prod_df = session.sql(prod_sql).to_pandas()

# Create 3-panel figure
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Panel 1: Environmental Conditions
ax1 = axes[0]
humidity_df = env_df[env_df['SENSOR_TYPE'] == 'HUMIDITY']
dust_df = env_df[env_df['SENSOR_TYPE'] == 'PM25']

ax1.plot(humidity_df['READING_DATE'], humidity_df['AVG_VALUE'], 
         color=SNOWFLAKE_BLUE, linewidth=2, marker='o', markersize=5, label='Humidity (%)')
ax1_twin = ax1.twinx()
ax1_twin.plot(dust_df['READING_DATE'], dust_df['AVG_VALUE'], 
              color=VALENCIA_ORANGE, linewidth=2, marker='s', markersize=5, label='Dust (µg/m³)')

# Crisis period shading
ax1.axvspan(pd.Timestamp('2024-12-09'), pd.Timestamp('2024-12-11'), 
            alpha=0.2, color='red', label='Crisis Period')

ax1.set_xlabel('Date')
ax1.set_ylabel('Humidity (%)', color=SNOWFLAKE_BLUE)
ax1_twin.set_ylabel('Dust PM2.5 (µg/m³)', color=VALENCIA_ORANGE)
ax1.set_title('Environmental Conditions')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='upper left')
ax1_twin.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Panel 2: AGV Error Rate
ax2 = axes[1]
bars = ax2.bar(agv_df['EVENT_DATE'], agv_df['ERROR_RATE_PCT'], 
               color=[VALENCIA_ORANGE if d >= pd.Timestamp('2024-12-09') and d <= pd.Timestamp('2024-12-11') 
                      else MID_BLUE for d in agv_df['EVENT_DATE']])
ax2.set_xlabel('Date')
ax2.set_ylabel('Error Rate (%)')
ax2.set_title('AGV-ERR-99 Error Rate')
ax2.tick_params(axis='x', rotation=45)
ax2.axhline(y=1, color=ACCENT_CYAN, linestyle='--', alpha=0.5, label='Normal (<1%)')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Panel 3: Production Completion by Line
ax3 = axes[2]
pivot_df = prod_df.pivot(index='PRODUCTION_LINE_ID', columns='PERIOD', values='COMPLETION_RATE').fillna(0)
x = range(len(pivot_df))
width = 0.35

if 'Normal' in pivot_df.columns:
    ax3.bar([i - width/2 for i in x], pivot_df['Normal'], width, 
            label='Normal Period', color=SNOWFLAKE_BLUE)
if 'Crisis' in pivot_df.columns:
    ax3.bar([i + width/2 for i in x], pivot_df['Crisis'], width, 
            label='Crisis Period', color=VALENCIA_ORANGE)

ax3.set_xlabel('Production Line')
ax3.set_ylabel('Completion Rate (%)')
ax3.set_title('Production Completion Rate')
ax3.set_xticks(x)
ax3.set_xticklabels(pivot_df.index)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')
ax3.set_ylim(0, 110)

plt.tight_layout()
plt.savefig('/tmp/crisis_verification.png', dpi=150, bbox_inches='tight', 
            facecolor='#121212', edgecolor='none')
plt.show()

print("\n✅ Visualization complete - crisis patterns verified")
print("   📉 Humidity dropped from ~45% to ~25% during crisis")
print("   📈 Dust spiked from ~10 to ~35 µg/m³")  
print("   ⚠️  AGV-ERR-99 rate increased to ~15% during crisis")
print("   🏭 LINE_4 completion dropped to ~65% during crisis")


## 12. Key Takeaways

### Generated Data Summary

| Data Type | Volume | Crisis Impact |
|-----------|--------|---------------|
| Environmental sensors | ~1.5M rows | Humidity 45% → 25%, Dust 10 → 35 µg/m³ |
| AGV telemetry | ~2.6M rows | 15% error rate during crisis (vs <1% normal) |
| Equipment states | ~11K rows | 60% starvation on LINE_4 during crisis |
| Production orders | ~1.4K rows | 60-70% completion during crisis |

### Causal Chain Demonstrated

```
Low Humidity (25%) → High Dust (35 µg/m³) → AGV-ERR-99 → Line Starvation → OPE Drop (60%)
```

This chain demonstrates the **OEE vs OPE gap**: 
- OEE measures equipment availability (unaffected by AGV issues)
- OPE measures operational performance (includes material flow dependencies)

### Interpretation Guidelines

| Metric | Normal Range | Crisis Range | Interpretation |
|--------|--------------|--------------|----------------|
| Humidity | 40-50% | 20-30% | Low humidity indicates HVAC issues or dry weather |
| PM2.5 Dust | 5-15 µg/m³ | 30-40 µg/m³ | High dust causes optical sensor failures |
| AGV Error Rate | <1% | 10-20% | ERR-99 indicates sensor obscuration |
| Completion Rate | 95-100% | 60-80% | Low completion = material starvation |

### Limitations

1. **Simplified Causal Model**: Real manufacturing has more complex failure modes
2. **Concentrated Crisis**: 3-day window (Dec 9-11) for demo clarity
3. **Uniform AGV Behavior**: All AGVs affected equally during crisis
4. **No Cascading Effects**: LINE_4 is independently worst-affected

### Next Steps

1. Run `03b_raw_to_atomic.sql` to populate ATOMIC layer tables
2. Execute OPE calculation views to compute metrics
3. Use the Streamlit dashboard to visualize the OEE-OPE gap
4. Train anomaly detection models on this labeled dataset
